# **Tworzenie API we Flasku – Wprowadzenie**

W tym ćwiczeniu nauczysz się, jak stworzyć proste API w Flasku, uruchomić je, wysyłać do niego zapytania oraz wykorzystać model decyzyjny w oparciu o podstawową regułę logiczną.

## **1️⃣ Tworzenie podstawowego API**
Najpierw utworzymy podstawową aplikację Flask.

### **Zapisanie kodu API do pliku**
W Jupyter Notebooku użyj magicznej komendy `%%file`, aby zapisać kod podstawowej aplikacji flask do pliku `app.py`: Kod znajdziesz na [cw1](https://sebkaz-teaching.github.io/RTA_2025/cw1.html)
Jako tekst do wyświetlenie strony głównej użyj `Witaj w moim API!`.

In [1]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Hello, World!"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


Teraz uruchom API w terminalu, wpisując:
```sh
python app.py
```
Flask uruchomi serwer lokalnie pod adresem `http://127.0.0.1:5000/`.

### **Sprawdzenie działania API**
W Jupyter Notebooku wykonaj zapytanie GET do strony głównej. Na podstawie pola `status_code` napisz wyrażenie warunkowe które dla status_code 200 wyświetli zawartość odpowiedzi (z pola `content`).

In [2]:
import requests
#response = pass TWOJ KOD

adres_url = "http://127.0.0.1:5000/"
response = requests.get(adres_url)

if response.status_code == 200:
    print(response.content.decode())
else: 
    print("Pobranie content się nie udało, status code: ", response.status_code)

{"message":"Hello, World!"}



Jeśli wszystko działa poprawnie, zobaczysz komunikat `Witaj w moim API!`.
---

## **2️⃣ Dodanie nowej podstrony**
Dodajmy nową podstronę `mojastrona`, która zwróci komunikat `To jest moja strona!`.

In [7]:
%%file app.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def main_page():
    return "Witaj w moim API!"

@app.route('/mojastrona')
def my_page():
    return "To jest moja strona!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


Ponownie uruchom API i wykonaj zapytanie do strony `"http://127.0.0.1:5000/mojastrona"`:

In [8]:
#response = pass 
import requests

adres_url = "http://127.0.0.1:5000/mojastrona"
response = requests.get(adres_url)

if response.status_code == 200:
    print(response.content.decode())
else: 
    print("Pobieranie się nie udało, status code: ", response.status_code)

To jest moja strona!


Powinieneś zobaczyć: `To jest moja strona!`

---

## **3️⃣ Automatyczne uruchamianie serwera z Jupyter Notebook**
Zamknij wcześniej uruchomiony serwer (`Ctrl+C` w terminalu) i uruchom go ponownie bezpośrednio z Jupyter Notebook, korzystając z `subprocess.Popen`:

In [9]:
import subprocess
p = subprocess.Popen(["python", "app.py"])
#server = pass

Po testach zamknij serwer wykorzystując metodę `kill`:


In [10]:
p.kill()

---

## **4️⃣ Obsługa parametrów w adresie URL**
Dodajemy nową podstronę `/hello`, która będzie przyjmować parametr `name`.

Edytuj `app.py`, dodając odpowiedni  kod


In [11]:
%%file app.py
from flask import Flask, request

# Create a flask
app = Flask(__name__)

# Create an API end point - strona glowna
@app.route('/')
def main_page():
    return "Witaj w moim API!"
    
# Create an API end point - moja strona
@app.route('/mojastrona')
def my_page():
    return "To jest moja strona!"
    
# Create an API end point - strona hello
@app.route('/hello', methods=['GET'])
def say_hello():
    name = request.args.get("name", "")
    resp = f"Hello {name}"
    return resp

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


In [13]:
import requests

res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content.decode())

res2 = requests.get("http://127.0.0.1:5000/hello?name=Dominika")
print(res2.content.decode())

Hello 
Hello Dominika


Uruchom serwer i sprawdź działanie API:
```python
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"
```

---

## **5️⃣ Tworzenie API z prostym modelem ML**
Stworzymy nową podstronę `/api/v1.0/predict`, która przyjmuje dwie liczby i zwraca wynik reguły decyzyjnej:
- Jeśli suma dwóch liczb jest większa niż 5.8, zwraca `1`.
- W przeciwnym razie zwraca `0`.


In [1]:
%%file app.py

from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

@app.route('/mojastrona')
def moja_strona():
    return "To jest moja strona!"

@app.route('/hello')
def hello():
    name = request.args.get('name') 
    if name:
        return f"Hello {name}!"
    else:
        return "Hello!"

@app.route('/api/v1.0/predict')
def predict():
    try:
        num1 = float(request.args.get('num1', 0))
        num2 = float(request.args.get('num2', 0))
        total = num1 + num2

        prediction = 1 if total > 5.8 else 0

        return jsonify({
            "prediction": prediction,
            "features": {
                "num1": num1,
                "num2": num2
            }
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 400

@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Not Found"}), 404

@app.errorhandler(400)
def bad_request(error):
    return jsonify({"error": "Bad Request"}), 400
    

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Overwriting app.py


In [3]:
import requests

res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())

{'features': {'num1': 3.0, 'num2': 4.0}, 'prediction': 1}


Sprawdź działanie API:
```python
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}
```

---

## **Podsumowanie**
Po wykonaniu tego ćwiczenia studenci będą umieli:
✅ Tworzyć podstawowe API w Flasku.  
✅ Dodawać podstrony i obsługiwać parametry URL.  
✅ Wysyłać zapytania GET i analizować odpowiedzi.  
✅ Automatycznie uruchamiać serwer z Jupyter Notebook.  
✅ Implementować prosty model decyzyjny w API.  

Gotowi na kolejne wyzwania? 🚀

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>

